Combine PKE textrank and word2vec

1. Imports & Setup

In [10]:
#1. rutin1 import module
import pandas as pd
import os
import sys
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
#2. rutin2 membuat syspath ke root utk aktifkan __init__.py
repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))
sys.path.append(repo_root)

2. Load Dataset

In [12]:
#3. rutin3 Load the dataset
dataset_path = os.path.join(repo_root, "notebooks/postager_nlp-id/dataset_ekstraksi_r29_pos_sm.xlsx")
df = pd.read_excel(dataset_path)
df["text"] = df["judul"] +". "+ df["isi"]
df_pos = df['pos_sentence_list']

In [13]:
# Preprocess
import re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemover, ArrayDictionary

def preprocess(text):
    text = re.sub(r"\b[a-zA-Z]\b", "", text)
    text = text.replace('.', '. ')
    text = re.sub('[^a-zA-Z.]', ' ', text)
    text = text.lower()
    text = re.sub("(\\d|\\W)+"," ",text)
    text = text.strip()

    stopwords_path = os.path.join(repo_root, "data/all_stop_words.txt")
    with open(stopwords_path, 'r') as f:
        stopwords = [line.strip() for line in f]

    dictionary = ArrayDictionary(stopwords)
    str = StopWordRemover(dictionary)
    text = str.remove(text)

    return text

df['preprocessed_text'] = df['text'].apply(preprocess)
df_tr = df['preprocessed_text'].apply(preprocess)

3. Process

In [14]:
import numpy as np
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

from nltk.util import ngrams

def generate_ngrams(words, n=2):
    """Generate ngrams from a list of words."""
    return [" ".join(gram) for gram in ngrams(words, n)]

def get_phrase_embedding(phrase, w2v_model):
    """Get the averaged word embedding for a phrase."""
    words = phrase.split()
    embeddings = [w2v_model.wv[word] for word in words if word in w2v_model.wv.key_to_index]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return None

def extract_keyphrases_with_ngrams(text, w2v_model):
    # Tokenize the text into unigrams
    unigrams = text.split()
    
    # Generate bigrams and trigrams
    bigrams = generate_ngrams(unigrams, 2)
    trigrams = generate_ngrams(unigrams, 3)
    
    # Combine unigrams, bigrams, and trigrams
    all_tokens = unigrams + bigrams + trigrams
    
    # Get embeddings for each token (averaging word embeddings for bigrams/trigrams)
    token_embeddings = [get_phrase_embedding(token, w2v_model) for token in all_tokens]
    
    # Filter out tokens that don't have embeddings
    tokens, embeddings = zip(*[(token, emb) for token, emb in zip(all_tokens, token_embeddings) if emb is not None])
    
    # Compute the cosine similarity between token embeddings
    cosine_matrix = cosine_similarity(embeddings)
    
    # Create a graph and connect tokens with high similarity
    G = nx.Graph()
    for i in range(len(tokens)):
        for j in range(len(tokens)):
            if cosine_matrix[i][j] > 0.5:  # This threshold can be adjusted
                G.add_edge(tokens[i], tokens[j], weight=cosine_matrix[i][j])
    
    # Compute the PageRank scores to rank the tokens
    scores = nx.pagerank(G)

    # Extract top N keyphrases along with their scores
    ranked_tokens = sorted(((scores[token], token) for token in tokens if token in scores), reverse=True)
    keyphrases_with_scores = [(token, score) for score, token in ranked_tokens[:3]]

    return keyphrases_with_scores

# Note: This is just the function definition for reference.
# You can use this updated function in your environment to process the dataframe.



In [15]:
w2v_path = os.path.join(repo_root, "models/w2v/idwiki_word2vec_100_new_lower.model")
w2v_model = Word2Vec.load(w2v_path)

single text

In [16]:
# Assuming w2v_model is your loaded Word2Vec model
text = df_tr[1]
keyphrases = extract_keyphrases_with_ngrams(text, w2v_model)
print(keyphrases)

[('acuan pengelolaan atas', 0.023077737722580618), ('demikian acuan pengelolaan', 0.022917046023445624), ('processing facilities', 0.022451316561733103)]


multitext

In [17]:
# Assuming df_pos is your dataframe with texts
results = []

# Iterate over the dataframe
for text in df_pos:
    keyphrases_with_scores = extract_keyphrases_with_ngrams(text, w2v_model)
    results.extend(keyphrases_with_scores)

# Convert the results to a dataframe
predict_textrank = pd.DataFrame(results, columns=['Keyword', 'Score'])
predict_textrank.head()


ValueError: not enough values to unpack (expected 2, got 0)